In [1]:
import networkx as nx
import numpy as np
import random
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_curve, auc
from sklearn.model_selection import cross_val_predict

### Convert gml to edgelist.csv. Optional.

In [2]:
def gml_to_edgelist(f):
    g = nx.read_gml(f)
    nx.write_edgelist(g, 'power.csv', delimiter=',')


In [3]:
def load_data(path):
    g = nx.read_edgelist(path, delimiter=',')
    return g

def generate_non_edge_list(g):
    n = len(g.edges()) # Negative edge list size is same as positive list size
    non_edges = []
    for u in g.nodes():
        for v in g.nodes():
            if u == v: continue
            if g.has_edge(u, v): continue
            non_edges.append((u, v))
    neg_sample = random.sample(non_edges, n)
    return neg_sample

def generate_class_labels(g, edges):
    y = []
    for edge in edges:
        if g.has_edge(edge[0], edge[1]):
            y.append(1)
        else:
            y.append(0)
    return y

### Implement features

In [4]:
def adamic_adar(g, X):
    preds = nx.adamic_adar_index(g, X)
    lst = []
    for u, v, p in preds:
        lst.append(p)

    max_p = max(lst)
    return [x/max_p for x in lst]

def jaccard(g, X):
    preds = nx.jaccard_coefficient(g, X)
    lst = []
    for u, v, p in preds:
        lst.append(p)

    max_p = max(lst)
    return [x/max_p for x in lst]

def common_neighbors(g, X):
    lst = []
    for x in X:
        cn = nx.common_neighbors(g, x[0], x[1])
        lst.append(len(list(cn)))
    max_p = float(max(lst))

    return [x/max_p for x in lst]

In [5]:
def concat_features(X, features):
    lst = [[] for x in range(len(X))]
    for feature in features:
        for i in range(len(X)):
            feature_value = feature[i]
            lst[i].append(feature_value)
    return lst

### Load data

In [6]:
g = load_data('power.csv')

### List of positive edges

In [14]:
EDGES_POSITIVE = list(g.edges())

<class 'list'>


### List of negative edges

In [12]:
EDGES_NEGATIVE = generate_non_edge_list(g)

<class 'list'>


In [15]:
EDGES = EDGES_POSITIVE + EDGES_NEGATIVE
random.shuffle(EDGES)

Y = generate_class_labels(g, EDGES)

feature1 = adamic_adar(g, EDGES)
feature2 = jaccard(g, EDGES)
feature3 = common_neighbors(g, EDGES)

features = [feature1, feature2, feature3]
feature_values = concat_features(EDGES, features)

In [18]:
print ("Total nodes", len(g.nodes()))
print ("Total edges", len(g.edges()))

Total nodes 4941
Total edges 6594


### Training

In [19]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=1)
# clf = svm.SVC(kernel='rbf', random_state=0, gamma=1, C=1)
clf.fit(feature_values, Y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [20]:
### Validation

In [21]:
pred = cross_val_predict(clf, feature_values, Y, cv=6)

### Results

In [22]:
print ("Accuracy:", accuracy_score(Y, pred))
precision, recall, fscore, support = precision_recall_fscore_support(Y, pred, average='binary')
print ("Precision:", precision)
print ("Recall:", recall)
print ("f-score:", fscore)

Accuracy: 0.603427358204
Precision: 0.994920174165
Recall: 0.207916287534
f-score: 0.343953838435
